In [82]:
# setup
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import requests
import calendar
import geopandas as gpd
import os.path as os
import scipy.stats
import seaborn.palettes
import seaborn.utils
import sys
from census import Census
from us import states
import http.client, urllib.request, urllib.parse, urllib.error, base64
#import config
import quickstart

root= r"C:/Users/Jennah/Desktop/Code/machine-learning-final"
inp= os.join(root, "data", "2_intermediate")
out= os.join(root, "data", "3_clean")

In [83]:
from pydrive.auth import GoogleAuth

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # Creates local webserver and auto handles authentication.

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=570719947456-53haph7mjc317oel1ujh0kmjbfd499jf.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [84]:
# files in clean data google drive
from pydrive.drive import GoogleDrive
drive = GoogleDrive(gauth)
fileList = drive.ListFile({'q': "'1jxSI_nq32-W9kPHKvQkzuNGJ5aIBD2j2' in parents and trashed=false"}).GetList()
for file in fileList:
  print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "To Share"):
      fileID = file['id']

Title: analysis_box.csv, ID: 1Vzjvta98IG9Hg4SfAGcd8JD8FaF3IXwy
Title: analysis_bbl_noacs.csv, ID: 1h-tDpBSslfzqzsJe1ny_ujDnr68RKry3
Title: analysis_bbl.csv, ID: 1yYX-ROr1c7fzcQiG6NZ1sh7Ko5gqr_4G


In [85]:
# files in intermediate data
from pydrive.drive import GoogleDrive
drive = GoogleDrive(gauth)
fileList = drive.ListFile({'q': "'1kiYbPtSYavHQKzyMWtXaLhP-JM2toXRm' in parents and trashed=false"}).GetList()
for file in fileList:
  print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "To Share"):
      fileID = file['id']

Title: pluto_df.csv, ID: 1j2KR6Tpa4CTA0k1omVfE52UYlMBMz17o
Title: hpd_agg_final.csv, ID: 1nIQD_4GDpg5ldTGgE5XqfiD11mxjrvEH
Title: ct_box_xwalk.csv, ID: 1nW-ODlhdUFKhTYzHZQOu3CCZN87oYXel
Title: fire_box_sum.csv, ID: 1zYSWlWtgNWsruaEkSSCqyXHIIdq7BEAP
Title: acs_2009_2019.csv, ID: 17PvjIPesTzugausfzMi7rTiXahTiHaH1
Title: census_acs, ID: 1DjJ3pOLAo4MOpgmeNPSXJbRzF_S10fDM
Title: weighted_borobox.csv, ID: 1H8yM7UH14YdMqi5ASjHMd_dwA4C7-lKe
Title: hpd_per_bbl.csv, ID: 1xXilfI1zEmyEYE_TtMdawcnmnZlTv__Z
Title: hpd_2012_2022_v3.csv, ID: 1HXst8E2ZuPVKvKe_9jmSQwKYppOJjraD
Title: pluto_borobox.csv, ID: 1Yf4xIIOBmKU2WIlImat8qaN3z2L239Nw
Title: fire_dispatch_tracts.csv, ID: 1psGrpBmGmNHPwM4lsRyiDX9tX5cf_UsO
Title: pluto_borobox.geojson, ID: 1UGelHNCzfSwsvbqBzyBX5GkrY_PmPGdp
Title: fire_dispatch.csv, ID: 1ulJlwpaezzq_aYV6jZzRWpmXLVlpO8qu
Title: dev_permits.csv, ID: 1mH_X0bLa_a1r-hpzwz43HpC7fFcayufp
Title: all_vacate.csv, ID: 1WfnTEjOaDNEb3E2vurXS_XkxJ0J1isdN
Title: pluto_ct.csv, ID: 1ZLZZS79fi4Z0RFTNMR

# Load Fire Dispatch Data

In [86]:
fires = drive.CreateFile({'id': '1zYSWlWtgNWsruaEkSSCqyXHIIdq7BEAP'})
fires.GetContentFile('fire_dispatch.csv')  

df_fires= pd.read_csv('fire_dispatch.csv').drop("Unnamed: 0", axis = 1)
df_fires.head()

,borobox,engines_assigned_quantity,ladders_assigned_quantity,dispatch_response_seconds_qy,num_fire_ev,avg_fire_ev_yr
0,B0015,0.0,0.000000,0.000000,1,0.0
1,B0016,0.0,0.000000,0.000000,1,0.0
2,B0018,3.0,2.000000,16.000000,1,1.0
3,B0020,3.5,2.333333,32.666667,6,1.2
4,B0023,3.0,2.000000,42.333333,3,3.0


## Load building level data

In [87]:
csv = drive.CreateFile({'id': '1yYX-ROr1c7fzcQiG6NZ1sh7Ko5gqr_4G'})
csv.GetContentFile('bld_predictors.csv')  

df_bld= pd.read_csv('bld_predictors.csv').drop("Unnamed: 0", axis = 1)
df_bld.head()

C:\Users\Jennah\anaconda3\envs\geo_env\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,borough,bbl,cd,tract2010,zipcode,address,bldgclass,landuse,ownertype,ownername,...,pctnhasian,pctnhhawainpi,pctnhother,pctnhtwoplus,pctownerocch,pctrentocch,crowding,crowdingrenter,geofips_recode,_merge
0,MN,1.012090e+09,107.0,181.0,10025.0,7 WEST 95 STREET,C2,2.0,NaN,ROBERT & ANNE RABBINO,...,6.191149,0.0,1.483051,2.106874,32.879925,67.120075,2.532833,3.773585,1018100.0,both
1,MN,1.012090e+09,107.0,181.0,10025.0,5 WEST 95 STREET,C5,2.0,NaN,"5 WEST 95TH STREET,",...,6.191149,0.0,1.483051,2.106874,32.879925,67.120075,2.532833,3.773585,1018100.0,both
2,MN,1.012090e+09,107.0,181.0,10025.0,12 WEST 96 STREET,D4,3.0,NaN,12 W 96TH STREET CORP,...,6.191149,0.0,1.483051,2.106874,32.879925,67.120075,2.532833,3.773585,1018100.0,both
3,MN,1.012090e+09,107.0,181.0,10025.0,44 WEST 96 STREET,D4,3.0,NaN,44 OWNERS CORP,...,6.191149,0.0,1.483051,2.106874,32.879925,67.120075,2.532833,3.773585,1018100.0,both
4,MN,1.012090e+09,107.0,181.0,10025.0,46 WEST 96 STREET,D4,3.0,NaN,46 W 96TH ST OWNERS CORP,...,6.191149,0.0,1.483051,2.106874,32.879925,67.120075,2.532833,3.773585,1018100.0,both


In [88]:
# drop missing bbl values
df_bld= df_bld.drop(df_bld[df_bld["bbl"].isna()].index, axis = 0)

In [89]:
## load borobox/bbl crosswalk
box = drive.CreateFile({'id': '1Yf4xIIOBmKU2WIlImat8qaN3z2L239Nw'})
box.GetContentFile('pluto_box.csv')  

df_box= pd.read_csv('pluto_box.csv').drop("Unnamed: 0", axis = 1)
df_box.head()

,bbl,ct2010,geom_point,location,latitude,zip,borobox,communitydistict,longitude,box_type,citycouncil,borough,dist
0,3042270037,1194.0,POINT (1018574.469941045 184812.8882210586),BELMONT AVE & MILFORD ST,40.673385,11208.0,B1826,BK05,-73.876800,ERS,37.0,Brooklyn,234.277634
1,3042670001,1208.0,POINT (1020385.163045117 183998.5145042881),BLAKE AVE & EUCLID AVE,40.671327,11208.0,B1838,BK05,-73.870925,ERS,42.0,Brooklyn,348.769018
2,3055400063,248.0,POINT (987459.9707733953 164891.168247966),20 AVE & 63 ST,40.617684,11204.0,B2859,BK11,-73.984652,ERS,47.0,Brooklyn,1198.302557
3,3029077506,477.0,POINT (1000672.538031318 199913.5662632599),MASPETH AVE & CONSELYEA ST,40.715519,11211.0,B0184,BK01,-73.941228,BARS,34.0,Brooklyn,139.662288
4,3055440004,250.0,POINT (985192.8136614707 166406.6672442852),15 AVE & 67 ST,40.622155,11219.0,B2805,BK11,-73.998333,ERS,38.0,Brooklyn,666.477552


In [90]:
df_bld_box= pd.merge(df_bld.drop("_merge", axis = 1), df_box[["bbl", "borobox", "dist"]], on = "bbl", indicator = True, how = "outer")

In [91]:
df_bld_box["_merge"].value_counts()

both          764716
left_only          0
right_only         0
Name: _merge, dtype: int64

In [92]:
# load predictors and predictors for acs
df_bld.columns
predictors = ["lotarea", "bldgarea", "numbldgs", "numfloors", "unitsres", "unitstotal", "assessland",
             "assesstot", "exempttot", "yearbuilt", "holc_AB", "holc_CD", "holc_D",
             "tot_hpd_vio", "hpd_fire_vio", "hpd_b_c_vio",
             "viol_count_per_unit", "hpd_fire_vio_per_unit", "hpd_b_c_vio_per_unit",
             "AL", "NB", "average_energy_usage", "average_energy_usage_mi"]
predictors_acs= df_bld.columns[df_bld.columns.get_loc("totpop"):df_bld.columns.get_loc("crowdingrenter") + 1].tolist()
predictors_full= predictors + predictors_acs
predictors_full

df_box_sum= df_bld_box.groupby("borobox", as_index = False)[predictors_full].mean()

## Merge to outcomes

In [93]:
df_box_final= pd.merge(df_box_sum, df_fires, on = "borobox", indicator = True, how = "left")
df_box_final.head()

,borobox,lotarea,bldgarea,numbldgs,numfloors,unitsres,unitstotal,assessland,assesstot,exempttot,...,pctownerocch,pctrentocch,crowding,crowdingrenter,engines_assigned_quantity,ladders_assigned_quantity,dispatch_response_seconds_qy,num_fire_ev,avg_fire_ev_yr,_merge
0,B0020,3457.956522,9239.391304,1.173913,3.521739,9.000000,9.043478,3.718826e+04,8.482943e+05,2.937391e+03,...,17.874693,82.125307,6.511057,6.806283,3.500000,2.333333,32.666667,6,1.200000,both
1,B0023,26081.875000,209572.375000,1.000000,18.750000,229.750000,231.125000,1.322048e+06,1.219295e+07,2.589945e+06,...,10.082423,89.917577,8.626047,7.280303,3.000000,2.000000,42.333333,3,3.000000,both
2,B0024,2754.847826,4832.108696,1.086957,3.217391,5.695652,5.847826,4.959654e+04,5.346010e+05,4.643715e+04,...,9.965636,90.034364,8.719931,7.156489,3.200000,2.040000,36.880000,25,1.923077,both
3,B0025,61653.000000,85292.250000,1.250000,10.750000,101.500000,102.000000,8.558175e+05,3.580575e+06,2.746800e+06,...,9.965636,90.034364,8.719931,7.156489,2.400000,1.600000,15.600000,5,1.666667,both
4,B0027,3888.574713,8242.965517,1.126437,3.183908,7.080460,7.402299,4.881548e+04,6.619610e+05,1.669940e+05,...,9.965636,90.034364,8.719931,7.156489,3.545455,2.340909,32.181818,44,2.933333,both


In [94]:
print(df_box_final.shape)
df_box_final["_merge"].value_counts()

(10864, 81)


both          10864
left_only         0
right_only        0
Name: _merge, dtype: int64

In [109]:
## investigate missing values
df_box_final.loc[df_box_final["medianhhincome"].isna(), "borobox":]

,borobox,lotarea,bldgarea,numbldgs,numfloors,unitsres,unitstotal,assessland,assesstot,exempttot,...,pctownerocch,pctrentocch,crowding,crowdingrenter,engines_assigned_quantity,ladders_assigned_quantity,dispatch_response_seconds_qy,num_fire_ev,avg_fire_ev_yr,_merge
1118,B1700,2573.210526,1723.289474,1.631579,2.052632,1.421053,1.421053,14273.684211,35038.421053,3172.368421,...,38.271605,61.728395,8.994709,12.285714,1.000000,1.000000,66.000000,2,1.000000,both
1143,B1734,3469.525000,1813.725000,1.425000,1.987500,1.750000,1.750000,15318.000000,43987.500000,552.500000,...,38.271605,61.728395,8.994709,12.285714,1.833333,1.500000,41.166667,6,2.000000,both
1167,B1758,2903.000000,1853.639344,1.737705,2.290984,1.393443,1.393443,15758.360656,45115.081967,3144.196721,...,38.271605,61.728395,8.994709,12.285714,2.392857,1.785714,53.678571,28,2.333333,both
3810,Q1447,7964.285714,2788.285714,1.857143,1.928571,1.000000,1.000000,27685.714286,85740.000000,2748.571429,...,100.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,1,0.000000,both
7980,R0259,2275.372881,2340.288136,1.101695,2.389831,2.186441,2.610169,19410.000000,66708.305085,6384.677966,...,19.128329,80.871671,10.653753,11.077844,3.241379,2.310345,30.379310,29,2.416667,both
7981,R0261,3771.600000,2233.600000,1.327273,2.436364,1.909091,1.927273,10856.727273,45471.272727,1278.000000,...,19.128329,80.871671,10.653753,11.077844,2.687500,1.812500,48.125000,16,1.454545,both
7982,R0262,2704.244444,1946.022222,1.133333,2.255556,2.022222,2.088889,12173.333333,47604.666667,1873.711111,...,19.128329,80.871671,10.653753,11.077844,2.857143,1.857143,45.785714,14,2.333333,both
7983,R0263,4978.000000,2087.520000,1.440000,2.200000,1.840000,1.920000,11934.000000,47692.800000,2126.800000,...,19.128329,80.871671,10.653753,11.077844,2.571429,1.857143,36.142857,14,2.000000,both


## Export analytic file

In [95]:
df_box_final.to_csv(os.join(root, "data", "3_clean", "analysis_box.csv"))